In [3]:
%load_ext autoreload
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [4]:
import sys
sys.path.append('../param_search')
import param_search as ps

In [5]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT
nvidia-smi

data_root=../../data/{data_name}
data_root=$(realpath $data_root)

case_name=Case{case_id:d}Pack
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment
output_dir=$case_dir/CorrField

mkdir -p $output_dir

fixed_name=case{case_id:d}_T{fixed_phase:02d}
fixed_image=$image_dir/$fixed_name.nii.gz
fixed_mask=$mask_dir/$fixed_name/lung_combined_mask.nii.gz

moving_name=case{case_id:d}_T{moving_phase:02d}
moving_image=$image_dir/$moving_name.nii.gz

reg_name=case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}
output_path=$output_dir/$reg_name

cd ../../../Lung250M-4B
python -m corrfield -F $fixed_image -M $moving_image -m $fixed_mask -O $output_path

echo Done
'''
name_format = 'register_{data_name}_case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}'

In [6]:
param_space = ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    fixed_phase=[50],
    moving_phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
)

for p in param_space:
    print(name_format.format(**p))
    
print(len(param_space))

register_Emory-4DCT_case1_T00_T50
register_Emory-4DCT_case1_T10_T50
register_Emory-4DCT_case1_T20_T50
register_Emory-4DCT_case1_T30_T50
register_Emory-4DCT_case1_T40_T50
register_Emory-4DCT_case1_T50_T50
register_Emory-4DCT_case1_T60_T50
register_Emory-4DCT_case1_T70_T50
register_Emory-4DCT_case1_T80_T50
register_Emory-4DCT_case1_T90_T50
register_Emory-4DCT_case2_T00_T50
register_Emory-4DCT_case2_T10_T50
register_Emory-4DCT_case2_T20_T50
register_Emory-4DCT_case2_T30_T50
register_Emory-4DCT_case2_T40_T50
register_Emory-4DCT_case2_T50_T50
register_Emory-4DCT_case2_T60_T50
register_Emory-4DCT_case2_T70_T50
register_Emory-4DCT_case2_T80_T50
register_Emory-4DCT_case2_T90_T50
register_Emory-4DCT_case3_T00_T50
register_Emory-4DCT_case3_T10_T50
register_Emory-4DCT_case3_T20_T50
register_Emory-4DCT_case3_T30_T50
register_Emory-4DCT_case3_T40_T50
register_Emory-4DCT_case3_T50_T50
register_Emory-4DCT_case3_T60_T50
register_Emory-4DCT_case3_T70_T50
register_Emory-4DCT_case3_T80_T50
register_Emory

In [ ]:
jobs = ps.submit(template, name_format, param_space, work_dir='2023-01-15_register')
jobs.to_csv('2023-01-15_register.jobs')
jobs

In [7]:
import pandas as pd
jobs = pd.read_csv('2023-01-15_register.jobs', index_col=0)
status = ps.status(jobs)
status

,index,data_name,case_id,fixed_phase,moving_phase,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,,
21681668,0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681669,1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681670,2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681671,3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681672,4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21681763,95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681764,96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681765,97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...


In [8]:
print(status.iloc[0].stderr)

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/ocean/projects/asc170022p/mtragoza/Lung250M-4B/corrfield/mindssc.py:46: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  F.conv3d(rpad(img), mshift1, dilation=delta) -
/ocean/projects/asc170022p/mtragoza/Lung250M-4B/corrfield/graphs.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /op

## Check output files

In [1]:
import numpy as np
import xarray as xr
import torch
import nibabel as nib

disp_file = 'data/Emory-4DCT/Case1Pack/CorrField/case1_T00_T50.nii.gz'
disp = torch.from_numpy(
    nib.load(disp_file).get_fdata().astype(np.float32)
)
disp.shape

torch.Size([1, 256, 256, 94, 3])

In [7]:
%autoreload
import data

ds = data.Emory4DCTDataset(data_root='data/Emory-4DCT', case_names=['Case1Pack'])
ds.load_images()
ds.load_masks(roi='lung_combined_mask')
ds.load_disps(fixed_phase=50)

Loading data/Emory-4DCT/Case1Pack/Images/case1_T00_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T10_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T20_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T30_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T40_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T50_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T60_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T70_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T80_s.img
Loading data/Emory-4DCT/Case1Pack/Images/case1_T90_s.img
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T00/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T10/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T20/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T30/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T40/lung_combined_mask.nii.gz
Loading data/Em

In [17]:
data.view_array(ds[0].array, groupby=['phase', 'y'], x='x', y='z')

:DynamicMap   [phase,y]
   :Image   [x,z]   (case1)

In [15]:
data.view_array(ds[0].disp, groupby=['phase', 'y', 'component'], x='x', y='z', clim=(-10, 10), cmap='seismic')

:DynamicMap   [phase,y,component]
   :Image   [x,z]   (displacement)

In [18]:
data.view_array(ds[0].mask, groupby=['phase', 'y'], x='x', y='z')

:DynamicMap   [phase,y]
   :Image   [x,z]   (mask)